In [ ]:
import os
import sys
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")

os.chdir("../../")
# sys.path.append()

In [ ]:
from gradsflow.data.image import image_dataset_from_directory
from gradsflow.core.data import AutoDataset
from gradsflow import Model
import timm

import torch
import torchvision

In [ ]:
data_dir = (
    "/Users/aniket/personal/gradsflow/gradsflow/data"  # replace with your filepath
)

In [ ]:
train_data = image_dataset_from_directory(
    f"{data_dir}/hymenoptera_data/train/",
    image_size=(96, 96),
    num_workers=None,
    transform=True,
    batch_size=4,
    shuffle=True,
)

val_data = image_dataset_from_directory(
    f"{data_dir}/hymenoptera_data/val/",
    image_size=(96, 96),
    num_workers=None,
    transform=True,
)

train_dataset = train_data["ds"]
train_dl = train_data["dl"]
val_dl = val_data["dl"]
num_classes = len(train_dataset.classes)
autodataset = AutoDataset(train_dl, val_dl, num_classes=num_classes)

In [ ]:
cnn = timm.create_model("ssl_resnet18", pretrained=True, num_classes=2)
model = Model(cnn, "adam")

In [ ]:
model.fit(autodataset, epochs=4, steps_per_epoch=1,)

Output()

┏━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ epoch ┃ train/loss ┃ val/loss ┃
┡━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 3     │  0.972     │  0.784   │
└───────┴────────────┴──────────┘

Finished Training


Tracker(max_epochs=4, epoch=3, steps_per_epoch=1, train_loss=0.9719440634159292, train_accuracy=None, val_loss=0.8289798090021954, val_accuracy=0.5620915032679739, val_steps=153, train_steps=2)